In [4]:
import pandas as pd

In [5]:
import numpy as np

In [6]:
import nltk

In [7]:
from nltk.stem import PorterStemmer

In [8]:
from nltk.tokenize import word_tokenize

In [9]:
from nltk.corpus import stopwords

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
from sklearn.preprocessing import OneHotEncoder

In [12]:
from sklearn.linear_model import LogisticRegression

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
from gensim.models import Word2Vec, KeyedVectors

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anton\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
modelo = KeyedVectors.load_word2vec_format("/git/dados/skip_s100/skip_s100.txt")

In [17]:
stop_words = stopwords.words('english')

In [18]:
# textos = []
# sentimentos = []

In [19]:
# arquivo = open("../../../dados/nlp/news_sentiment_analysis.csv", "r", encoding="utf-8")
# arquivo.readline()   # Descartando a primeira linha (header)
# linha = arquivo.readline()
# while linha != "":
#     colunas = linha.split(",")
#     texto = colunas[3]
#     sentimento = colunas[6]
#     textos.append(texto)
#     sentimentos.append(sentimento)
#     linha = arquivo.readline()
# arquivo.close()
    

In [28]:
df_news = pd.read_csv("../../../dados/nlp/news_sentiment_analysis.csv", sep=",", quotechar='"', encoding="utf-8")

In [29]:
df_news = df_news[df_news['Sentiment'] != "neutral"]

In [30]:
df_news["Sentiment_number"] = df_news['Sentiment'].apply(lambda x: 0 if x == 'negative' else 1)

In [31]:
# df_news["description_lower"] = df.news['Description'].apply(lambda x: x.lower())
df_news["description_lower"] = df_news['Description'].str.lower()

In [32]:
def remove_stopwords( texto ):
    tokens = word_tokenize( texto )
    nova_lista = []
    for token in tokens:
        if token not in stop_words:
            # nova_lista.append(ps.stem(token))
            nova_lista.append(token)
    return " ".join(nova_lista)
    

In [33]:
df_news["description_lower"][0]

'st. george — kaitlyn larson, a first-year teacher at pine view high school, recently received the best in state award for a business leadership course. larson’s students in grades 10-12 had the highest passing rate on the youscience exam during the spring and fall semesters of 2023. the youscience exam is similar to final exams [&#8230;]'

In [34]:
df_news["description_stem"] = df_news["description_lower"].apply(lambda x: remove_stopwords(x))

In [35]:
df_news["description_stem"]

0       st. george — kaitlyn larson , first-year teach...
2       ( marketscreener.com ) billionaire elon musk d...
3       ( marketscreener.com ) u.s. trade regulator fr...
4       4.5 million households u.s. solar panels homes...
5       ( marketscreener.com ) billionaire investor ma...
                              ...                        
3493    qrg capital management inc. trimmed position s...
3495    qrg capital management inc. increased stake sh...
3496    qrg capital management inc. bought new positio...
3497    qrg capital management inc. boosted stake akam...
3499    qrg capital management inc. reduced holdings s...
Name: description_stem, Length: 2711, dtype: object

In [ ]:
#vetorizador = CountVectorizer(max_features=2000)
# vetorizador = CountVectorizer()

In [ ]:
# bow_x = vetorizador.fit_transform(df_news["description_stem"])

In [ ]:
# vocabulario = vetorizador.get_feature_names_out()
# len(vocabulario)

In [184]:
# x_train, x_test, y_train, y_test = train_test_split(bow_x, df_news["Sentiment_number"], random_state=50)

In [185]:
# one_hot_train = pd.DataFrame.sparse.from_spmatrix(x_train, columns=vocabulario)
# one_hot_test = pd.DataFrame.sparse.from_spmatrix(x_test, columns=vocabulario)

In [52]:
def texto_to_embedding( texto ):
    tokens = texto.split(" ")
    # modelo_padrao = modelo["desconhecido"]
    lista_embeddings = []
    for token in tokens:
        if token in modelo:
            lista_embeddings.append(modelo[token])
    array_embeddings = np.array(lista_embeddings)
    return [array_embeddings.sum()]
        

In [53]:
# texto_to_embedding( df_news["description_stem"][0] )
df_news["description_final"] = df_news["description_stem"].apply(lambda x: texto_to_embedding(x))

In [54]:
df_news["description_final"]

0        [36.74553]
2        [67.08336]
3       [41.691902]
4       [105.82489]
5        [72.46743]
           ...     
3493     [36.29739]
3495     [42.85654]
3496     [51.72785]
3497    [41.106922]
3499     [54.08328]
Name: description_final, Length: 2711, dtype: object

In [55]:
x_train, x_test, y_train, y_test = train_test_split(df_news["description_final"], df_news["Sentiment_number"], random_state=50)

In [56]:
model = LogisticRegression()

In [57]:
model.fit(x_train, y_train)

ValueError: setting an array element with a sequence.

In [188]:
model.score(one_hot_test, y_test)

0.9100294985250738

### Score com todas palavras do vocabulario ==> 0.9129793510324484
### Score com 2000 palavras no vocabulario ==> 0.9174041297935103

In [76]:
novo_vetorizador = CountVectorizer(vocabulary=vocabulario)

In [83]:
bow_x_predict = novo_vetorizador.fit_transform(["The Baltimore Business Journal has hired Ben Terzi as its research editor. Terzi manages various databases to produce weekly Lists, the Book of Lists and other research projects for the Baltimore Business Journal. Prior to joining the Business Journal, he covered community news for The Dundalk Eagle and Avenue News. Born and raised in Baltimore, Terzi is"])

In [84]:
x_predict = pd.DataFrame.sparse.from_spmatrix(bow_x_predict, columns=vocabulario)

In [85]:
model.predict(x_predict)

array([1], dtype=int64)